In [51]:
import pandas as pd
import numpy as np
import scipy.signal as sng

In [52]:
et=pd.read_csv('извлеченные_айтрекинг/et_S01.csv')

In [53]:
def THE_MOST_STUPID_FUNCTION(eeg_producta):
    list_eeg={}
    electrodes=['left_pupil', 'right_pupil']
    for i in electrodes:
        eeg_producta_i=np.delete(np.array(eeg_producta[i].split('[')), [0, 1])
        s=[]
        for j in range(len(eeg_producta_i)):
            vrem=eeg_producta_i[j].split(',')[:-1]
            vrem[-1]=vrem[-1][:-1]
            for k in range(len(vrem)):
                vrem[k]=float(vrem[k])
            s.append(vrem)
        list_eeg[f'{i}']=s
    return list_eeg

In [54]:
#ИЛИ НАДО СНОВА FFT ИЛИ PSD????#

def get_pupils(et):
    s={}
    for i in range(len(et)):
        et_producta=et.loc[i]
        krivie_ruki=THE_MOST_STUPID_FUNCTION(et_producta)
        pp=et['product_page'].loc[i]
        pn=et['product_number'].loc[i]

        s[f'{pp} {pn}']=[et['index'].loc[i], et['info'].loc[i], krivie_ruki]
    return s

In [55]:
check=get_pupils(et=et)

In [56]:
def average_pupil(et_got):
    pupils=['left_pupil', 'right_pupil']
    for i in et_got:
        et_page_prod=et_got[i][2]
        for k in pupils:
            vrem=[]
            for j in range(len(et_page_prod[k])):
                vrem.append(np.mean(et_page_prod[k][j]))
            avg_pupil=np.mean(vrem)
            check[i][2][f"averaged_{k}"]=round(avg_pupil, ndigits=5)

In [57]:
average_pupil(check)

In [58]:
def get_result(avg_pup):
    l=[]
    r=[]
    for i in avg_pup:
        l.append(avg_pup[i][2]['averaged_left_pupil'])
        r.append(avg_pup[i][2]['averaged_right_pupil'])
    return [l, r]

In [59]:
final=get_result(check)

In [60]:
df=pd.DataFrame({'product_page_number': list(check)})
df['left_pupil_avg']=final[0]
df['right_pupil_avg']=final[1]

In [61]:
df

,product_page_number,left_pupil_avg,right_pupil_avg
0,Page1 0,3.30437,3.23903
1,Page1 1,3.27006,3.20299
2,Page1 2,3.28092,3.22018
3,Page1 3,3.44064,3.34842
4,Page1 4,3.36216,3.29881
...,...,...,...
127,Page6 19,3.60834,3.49774
128,Page6 20,3.71160,3.60043
129,Page6 21,3.44980,3.34990
130,Page6 22,3.67248,3.51599


In [62]:
people=['01', '02', '03', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
df=pd.DataFrame()

for t in people:
    et=pd.read_csv(f'извлеченные_айтрекинг/et_S{t}.csv')
    check=get_pupils(et=et)
    average_pupil(check)
    final=get_result(check)

    df_people=pd.DataFrame({'people': np.full(len(list(check)), t), 'product_page_number': list(check)})
    df_people['left_pupil_avg']=final[0]
    df_people['right_pupil_avg']=final[1]

    df=pd.concat([df, df_people])

In [63]:
#df.to_csv('et_try_metrics.csv')

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5902 entries, 0 to 141
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   people               5902 non-null   object 
 1   product_page_number  5902 non-null   object 
 2   left_pupil_avg       5902 non-null   float64
 3   right_pupil_avg      5902 non-null   float64
dtypes: float64(2), object(2)
memory usage: 230.5+ KB
